<img src="https://github.com/rjpost20/Anomalous-Bank-Transactions-Detection-Project/blob/main/data/AdobeStock_319163865.jpeg?raw=true">
Image by <a href="https://stock.adobe.com/contributor/200768506/andsus?load_type=author&prev_url=detail" >AndSus</a> on Adobe Stock

# Phase 5 Project: *Detecting Anomalous Financial Transactions*

## Notebook 1: Intro, EDA and Preprocessing

### By Ryan Posternak

Flatiron School, Full-Time Live NYC<br>
Project Presentation Date: August 25th, 2022<br>
Instructor: Joseph Mata

## Goal: 

*This is a project for learning purposes. The *** is not involved with this project in any way.*

<br>

# Overview and Business Understanding

<br>

# Data Understanding

<br>

# Initial Exploratory Data Analysis

### Google colab compatibility downloads

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
!pip install pyspark==3
!pip install -q findspark
import findspark
findspark.init()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import libraries, packages and modules

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from itertools import chain
import os

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DoubleType, TimestampType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
from IPython.display import HTML, display
%matplotlib inline

In [ ]:
# Check colab GPU info

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

In [5]:
# Set text to wrap in Google colab notebook

def set_css():
    display(HTML('''
    <style>
      pre {
          white-space: pre-wrap;
      }
    </style>
    '''))
get_ipython().events.register('pre_run_cell', set_css)

In [4]:
# Initialize Spark Session

# spark = SparkSession.builder.master('local[*]').getOrCreate()
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

### Description of Features

**Dataset 1 – Transactions:**

`MessageId` - Globally unique identifier within this dataset for individual transactions<br>
`UETR` - The Unique End-to-end Transaction Reference—a 36-character string enabling traceability of all individual transactions associated with a single end-to-end transaction<br>
`TransactionReference` - Unique identifier for an individual transaction<br>
`Timestamp` - Time at which the individual transaction was initiated<br>
`Sender` - Institution (bank) initiating/sending the individual transaction<br>
`Receiver` - Institution (bank) receiving the individual transaction<br>
`OrderingAccount` - Account identifier for the originating ordering entity (individual or organization) for end-to-end transaction<br>
`OrderingName` - Name for the originating ordering entity<br>
`OrderingStreet` - Street address for the originating ordering entity<br>
`OrderingCountryCityZip` - Remaining address details for the originating ordering entity<br>
`BeneficiaryAccount` - Account identifier for the final beneficiary entity (individual or organization) for end-to-end transaction<br>
`BeneficiaryName` - Name for the final beneficiary entity<br>
`BeneficiaryStreet` - Street address for the final beneficiary entity<br>
`BeneficiaryCountryCityZip` - Remaining address details for the final beneficiary entity<br>
`SettlementDate` - Date the individual transaction was settled<br>
`SettlementCurrency` - Currency used for transaction<br>
`SettlementAmount` - Value of the transaction net of fees/transfer charges/forex<br>
`InstructedCurrency` - Currency of the individual transaction as instructed to be paid by the Sender<br>
`InstructedAmount` - Value of the individual transaction as instructed to be paid by the Sender<br>
`Label` - Boolean indicator of whether the transaction is anomalous or not. This is the target variable for the prediction task.<br>
<br>
**Dataset 2 – Banks:**

`Bank` - Identifier for the bank<br>
`Account` - Identifier for the account<br>
`Name` - Name of the account<br>
`Street` - Street address associated with the account<br>
`CountryCityZip` - Remaining address details associated with the account<br>
`Flags` - Enumerated data type indicating potential issues or special features that have been associated with an account. Flag definitions are below:<br>
00 - No flags<br>
01 - Account closed<br>
03 - Account recently opened<br>
04 - Name mismatch<br>
05 - Account under monitoring<br>
06 - Account suspended<br>
07 - Account frozen<br>
08 - Non-transaction account<br>
09 - Beneficiary deceased<br>
10 - Invalid company ID<br>
11 - Invalid individual ID<br>

### Read in Data

In [6]:
# Read in transactions data csv file to a Spark DataFrame
train_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/transaction_train_dataset.csv', header=True, inferSchema=True)
test_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/transaction_test_dataset.csv', header=True, inferSchema=True)

# Persist in memory
train_df = train_df.cache()
test_df = test_df.cache()

# Read in banks data csv file to a Spark DataFrame
# banks_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/bank_dataset.csv', header=True, inferSchema=True)

### Initial EDA

In [ ]:
# Print shape of dataframes
print(f"train_df:  {train_df.count():,} Rows, {len(train_df.columns)} Columns")
print(f"test_df:  {test_df.count():,} Rows, {len(test_df.columns)} Columns")
# print(f"banks_df:  {banks_df.count():,} Rows, {len(banks_df.columns)} Columns")

In [ ]:
transactions_df.printSchema()

In [ ]:
# banks_df.printSchema()

In [ ]:
# Display first 3 rows of transactions dataframe
transactions_df.show(n=3, vertical=True, truncate=False)

In [ ]:
# Display first 5 rows of banks dataframe
# banks_df.show(n=5, truncate=False)

In [ ]:
# Print number of null/missing values in each column of transactions_df
transactions_df_null = transactions_df.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c))\
                                               .alias(c) for c in transactions_df.columns])

print('Number of null/missing values per column:\n')
transactions_df_null.show(vertical=True, truncate=False)

In [ ]:
# Print number of null/missing values in each column of banks_df
# banks_df_null = banks_df.select([F.count(F.when(F.col(c).contains('None') | \
#                                                 F.col(c).contains('NULL') | \
#                                                 (F.col(c) == '' ) | \
#                                                 F.col(c).isNull() | \
#                                                 F.isnan(c), c ))\
#                                  .alias(c) for c in banks_df.columns])

# print('Number of null/missing values per column:\n')
# banks_df_null.show(vertical=True, truncate=False)

In [24]:
# Print number of unique values in each column in train_df; sample 1% of df for efficiency
train_df_unique = train_df.sample(False, 0.01).agg(*(F.countDistinct(F.col(c)) for c in train_df.columns))

print(f"Number of unique values per column (in sample of 1% of dataframe):\n")
train_df_unique.show(vertical=True, truncate=False)

Number of unique values per column (in sample of 1% of dataframe):

-RECORD 0---------------------------------
 count(MessageId)                 | 40784 
 count(Timestamp)                 | 18040 
 count(UETR)                      | 40784 
 count(Sender)                    | 15    
 count(Receiver)                  | 18    
 count(TransactionReference)      | 24155 
 count(OrderingAccount)           | 35742 
 count(OrderingName)              | 22416 
 count(OrderingStreet)            | 34234 
 count(OrderingCountryCityZip)    | 32114 
 count(BeneficiaryAccount)        | 29025 
 count(BeneficiaryName)           | 19625 
 count(BeneficiaryStreet)         | 26492 
 count(BeneficiaryCountryCityZip) | 26245 
 count(SettlementDate)            | 23    
 count(SettlementCurrency)        | 4     
 count(SettlementAmount)          | 40782 
 count(InstructedCurrency)        | 5     
 count(InstructedAmount)          | 40783 
 count(Label)                     | 2     
 count(Hour)                 

In [ ]:
# Print number of unique values in each column in banks_df; sample 10% of df for efficiency
# banks_df_unique = banks_df.sample(False, 0.1).agg(*(F.countDistinct(F.col(c)) for c in banks_df.columns))

# print(f"Number of unique values per column (in sample of 10% of dataframe):\n")
# banks_df_unique.show(vertical=True, truncate=False)

In [ ]:
# Display Pandas style summary statistics table of numeric columns
num_cols = [item[0] for item in transactions_df.dtypes if item[1] == 'int' or item[1] == 'double']

transactions_df.select(num_cols).summary().show(truncate=False)


In [22]:
# Display unique currencies used in transactions
print('train_df:')
train_df.select('InstructedCurrency').distinct().show()

train_df:
+------------------+
|InstructedCurrency|
+------------------+
|               EUR|
|               USD|
|               GBP|
|               JPY|
|               INR|
|               NZD|
|               CAD|
|               AUD|
+------------------+



In [23]:
# Display value counts for 'Label' column (classification target) in train_df
class_counts = train_df.groupBy('Label').count().withColumn('percent', F.col('count')/train_df.count())

class_counts.show(truncate=10)

+-----+-------+----------+
|Label|  count|   percent|
+-----+-------+----------+
|    0|4049426|0.99880...|
|    1|   4852|0.00119...|
+-----+-------+----------+



**Remarks:**
- It looks like this is an extremely imbalanced dataset - only about 0.1% of the data is in the positive class. We will need to address this class imbalance as part of the modeling process.

<br>

# Preprocessing & Feature Engineering

Steps:
1. Change column datatypes to correct type
2. Drop duplicate transaction rows
3. Train/test split
4. Create `SenderHourFreq` feature
5. Create `SenderCurrencyFreq` and `SenderCurrencyAmtAvg` features
6. Create `SenderReceiverFreq` feature

In [ ]:
# Specify join condition
# join_condition = (transactions_df.OrderingAccount == banks_df.Account) | (transactions_df.BeneficiaryAccount == banks_df.Account)

# Join dataframes
# preprocessed_df = transactions_df.join(banks_df, on=join_condition, how='left')

# Unpersist old dataframes from memory
# display(transactions_df.unpersist())
# display(banks_df.unpersist())

In [ ]:
# Print shape of joined dataframe
# print(f"{preprocessed_df.count():,} Rows, {len(preprocessed_df.columns)} Columns")

### Convert `Timestamp` column to TimestampType

In [7]:
# Convert 'Timestamp' column to TimestampType
train_df = train_df.withColumn('Timestamp', train_df.Timestamp.cast(TimestampType()))
test_df = test_df.withColumn('Timestamp', test_df.Timestamp.cast(TimestampType()))

assert train_df.select('Timestamp').dtypes[0][0] == 'Timestamp'
assert test_df.select('Timestamp').dtypes[0][0] == 'Timestamp'

### Drop intermediary transactions (only keep one row per end-to-end transaction)

In [8]:
# Print count of unique transactions (as identified by UETR codes)
print('train_df:')
train_df.select(F.countDistinct('UETR')).show()
print('test_df:')
test_df.select(F.countDistinct('UETR')).show()

train_df:
+--------------------+
|count(DISTINCT UETR)|
+--------------------+
|             4054278|
+--------------------+

test_df:
+--------------------+
|count(DISTINCT UETR)|
+--------------------+
|              651584|
+--------------------+



In [ ]:
# Print total number of combined rows with duplicate UETR values (meaning sender routed transaction through one or more intermediary banks)
transactions_df.select('UETR').groupBy('UETR')\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

In [9]:
# Drop rows with duplicate UETR codes, keeping the first occurence (sorted by Timestamp)
train_df = train_df.orderBy('Timestamp').coalesce(1).dropDuplicates(subset = ['UETR']).cache()
test_df = test_df.orderBy('Timestamp').coalesce(1).dropDuplicates(subset = ['UETR']).cache()

# Ensure no duplicates
assert train_df.groupBy(train_df.UETR).count().where(F.col('count') > 1).count() == 0
assert test_df.groupBy(test_df.UETR).count().where(F.col('count') > 1).count() == 0

print(f"train_df: {train_df.count()} rows")
print(f"test_df: {test_df.count()} rows")

train_df: 4054278 rows
test_df: 651584 rows


In [ ]:
# Show value counts for 'Label' column (classification target) in new train and test dataframes
class_counts_train = train_df.groupBy('Label').count().withColumn('percent', F.col('count')/train_df.count())
class_counts_test = test_df.groupBy('Label').count().withColumn('percent', F.col('count')/test_df.count())

print('train_df:')
class_counts_train.show(truncate=10)
print('test_df:')
class_counts_test.show(truncate=10)

## Feature Engineering

### Create `SenderHourFreq` feature: transaction hour frequency for each sender

This feature will tell us the frequency with which each sender initiated transactions for each hour of the day. Certain hours of the day are potentially correlated with more anomalous transactions.

In [10]:
# Define UDF to extract hour from timestamp
hour = F.udf(lambda x: x.hour, IntegerType())

# Create new column of transaction hours
train_df = train_df.withColumn('Hour', hour(train_df.Timestamp))
test_df = test_df.withColumn('Hour', hour(test_df.Timestamp))

# Create list of unique senders
senders = train_df.select('Sender').toPandas()['Sender'].unique()

# Create column of senders concatenated with hours
train_df = train_df.withColumn('SenderHour', F.concat(F.col('Sender'), F.col('Hour').cast(StringType())))
test_df = test_df.withColumn('SenderHour', F.concat(F.col('Sender'), F.col('Hour').cast(StringType())))

pd_df = train_df.select('Sender', 'Hour').toPandas()

# Create dictionary of sender hour frequency values to map from sender hour values
sender_hour_frequency = {}
for sender in senders:
    sender_rows = pd_df[pd_df['Sender'] == sender]
    for hour in range(24):
        sender_hour_frequency[sender + str(hour)] = len(sender_rows[sender_rows['Hour'] == hour])

# Create new column in train and test dataframes with sender_hour_frequency dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_hour_frequency.items())])

train_df = train_df.withColumn('SenderHourFreq', mapping_expr[F.col('SenderHour')])
test_df = test_df.withColumn('SenderHourFreq', mapping_expr[F.col('SenderHour')])

### Create `SenderCurrencyFreq` and `SenderCurrencyAmtAvg` features: transaction currency frequency and average transaction amount per currency for each sender

These features will tell us the frequency with which each sender initiated transactions for each currency, in the case of the first feature. For the second feature, it will tell us the average amount with which each sender sent each currency. These features may also be correlated with anomalous transactions.

In [11]:
# Create column of senders concatenated with instructed currencies
train_df = train_df.withColumn('SenderCurrency', F.concat(F.col('Sender'), F.col('InstructedCurrency')))
test_df = test_df.withColumn('SenderCurrency', F.concat(F.col('Sender'), F.col('InstructedCurrency')))

pd_train_df = train_df.select('SenderCurrency', 'InstructedAmount').toPandas()
pd_test_df = test_df.select('SenderCurrency', 'InstructedAmount').toPandas()

# Create dictionary of sender currency frequency values to map from sender currency values
sender_currency_freq = {}
# Create dictionary of average sender currency values to map from sender currency values
sender_currency_avg = {}

for sc in set(
    list(pd_train_df['SenderCurrency'].unique()) + list(pd_test_df['SenderCurrency'].unique())
):
    sender_currency_freq[sc] = len(pd_train_df[pd_train_df['SenderCurrency'] == sc])
    sender_currency_avg[sc] = pd_train_df[pd_train_df['SenderCurrency'] == sc][
        "InstructedAmount"
    ].mean()

# Create new column in train and test dataframes with sender_currency_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_currency_freq.items())])

train_df = train_df.withColumn('SenderCurrencyFreq', mapping_expr[F.col('SenderCurrency')])
test_df = test_df.withColumn('SenderCurrencyFreq', mapping_expr[F.col('SenderCurrency')])

# Create new column in train and test dataframes with sender_currency_avg dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_currency_avg.items())])

train_df = train_df.withColumn('SenderCurrencyAmtAvg', mapping_expr[F.col('SenderCurrency')])
test_df = test_df.withColumn('SenderCurrencyAmtAvg', mapping_expr[F.col('SenderCurrency')])

### Create `SenderReceiverFreq` feature: sender-receiver combination frequency for each sender and receiver

In [13]:
# Create column of senders concatenated with receivers
train_df = train_df.withColumn('SenderReceiver', F.concat(F.col('Sender'), F.col('Receiver')))
test_df = test_df.withColumn('SenderReceiver', F.concat(F.col('Sender'), F.col('Receiver')))

# Create dictionary of sender receiver frequency values to map from sender receiver values
sender_receiver_freq = {}

pd_train_df = train_df.select('SenderReceiver').toPandas()
pd_test_df = test_df.select('SenderReceiver').toPandas()

for sr in set(
    list(pd_train_df['SenderReceiver'].unique()) + list(pd_test_df['SenderReceiver'].unique())
):
    sender_receiver_freq[sr] = len(pd_train_df[pd_train_df['SenderReceiver'] == sr])

# Create new column in train and test dataframes with sender_receiver_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_receiver_freq.items())])

train_df = train_df.withColumn('SenderReceiverFreq', mapping_expr[F.col('SenderReceiver')])
test_df = test_df.withColumn('SenderReceiverFreq', mapping_expr[F.col('SenderReceiver')])

### Drop categorical columns

In [28]:
cols_to_drop = [
    'MessageId',
    'Timestamp',
    'UETR',
    'Sender',
    'Receiver',
    'TransactionReference',
    'OrderingAccount',
    'OrderingName',
    'OrderingStreet',
    'OrderingCountryCityZip',
    'BeneficiaryAccount',
    'BeneficiaryName',
    'BeneficiaryStreet',
    'BeneficiaryCountryCityZip',
    'SettlementDate',
    'SettlementCurrency',
    'InstructedCurrency',
    'SenderHour',
    'SenderCurrency',
    'SenderReceiver'
]

train_df = train_df.drop(*cols_to_drop)
test_df = test_df.drop(*cols_to_drop)

In [32]:
train_df.show(5)

+----------------+----------------+-----+----+--------------+------------------+--------------------+------------------+
|SettlementAmount|InstructedAmount|Label|Hour|SenderHourFreq|SenderCurrencyFreq|SenderCurrencyAmtAvg|SenderReceiverFreq|
+----------------+----------------+-----+----+--------------+------------------+--------------------+------------------+
|      2939777.86|      2626443.19|    0|  15|         50830|           1403717|1.8244330186688203E8|           1628506|
|   1.654746139E7|   1.654746139E7|    0|   5|         31856|            405122| 4.768777282929207E7|             56650|
|       927212.55|      1153679.92|    0|   5|           216|            274065|  1671495.4797866924|            274065|
|   6.226574191E7|   6.226574191E7|    0|  13|         28488|            405122| 4.768777282929207E7|            307197|
|      6128948.94|      6128948.94|    0|  18|         10097|            405122| 4.768777282929207E7|            307197|
+----------------+--------------

In [33]:
test_df.show(5)

+----------------+----------------+-----+----+--------------+------------------+--------------------+------------------+
|SettlementAmount|InstructedAmount|Label|Hour|SenderHourFreq|SenderCurrencyFreq|SenderCurrencyAmtAvg|SenderReceiverFreq|
+----------------+----------------+-----+----+--------------+------------------+--------------------+------------------+
|      1585724.74|    1.61477062E8|    0|  22|          4298|             21921|9.481250514661982E11|              4362|
|      2862743.17|      2862743.17|    0|  13|         30133|            529744|  1674064.4259055888|            261994|
|      5139383.79|      4130341.39|    0|  16|         45327|            218987|  3691762.8543079915|           1628506|
|      6107360.55|      5456410.75|    0|  11|        536485|           1403717|1.8244330186688203E8|           1628506|
|      3365712.14|      3365692.14|    0|  14|         25641|            529744|  1674064.4259055888|            267752|
+----------------+--------------